In [1]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train_modified import train
from utils import AttrDict
import pandas as pd

In [2]:
cudnn.benchmark = True
cudnn.deterministic = False

In [3]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [4]:
import sys
sys.path.insert(0, '..')
import easyocr



reader = easyocr.Reader(["en"])



opt = get_config("config_files/en_filtered_config.yaml")

# print(['[blank]']+list(opt.character))
# print(reader.converter.character)

# assert False
train(opt, amp=False, model=reader.recognizer, converter=reader.converter)

number : 0123456789
symbol : !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ €
lang_char : ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
experiment_name : en_filtered
train_data : all_data
valid_data : all_data/en_val
manualSeed : 1111
workers : 6
batch_size : 32
num_iter : 10
valInterval : 5
saved_model : 
FT : False
optim : False
lr : 0.0001
beta1 : 0.9
rho : 0.95
eps : 1e-08
grad_clip : 5
select_data : en_train
batch_ratio : 1
total_data_usage_ratio : 1.0
batch_max_length : 34
imgH : 64
imgW : 600
rgb : False
contrast_adjust : 0.0
sensitive : True
PAD : True
data_filtering_off : False
Transformation : None
FeatureExtraction : VGG
SequenceModeling : BiLSTM
Prediction : CTC
num_fiducial : 20
input_channel : 1
output_channel : 256
hidden_size : 256
decode : greedy
new_prediction : False
freeze_FeatureFxtraction : False
freeze_SequenceModeling : False
character : 0123456789!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ €ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Filtering the images containing char

AssertionError: 